# Structure gemini outputs with Pydantic

In [1]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents= "List a few asian soups recipes, a yummy description and list the ingredients"
)

print(response.text)

Here are a few iconic and delicious Asian soup recipes, complete with a yummy description and their key ingredients:

---

### 1. Pho Bo (Vietnamese Beef Noodle Soup)

**Yummy Description:**
An aromatic symphony in a bowl, Pho Bo is the soul-warming comfort food that transports you straight to the bustling streets of Vietnam. Imagine a rich, crystal-clear beef broth, patiently simmered for hours with fragrant spices like star anise, cinnamon, and ginger. This savory elixir cradles silky flat rice noodles, paper-thin slices of tender rare beef that cook in the hot broth, and often slices of well-done brisket or flank. Served with a vibrant platter of fresh herbs like Thai basil, mint, and cilantro, crunchy bean sprouts, zesty lime wedges, and fiery chili, each spoonful is a dance of complex flavors – sweet, savory, umami, and fresh. It's truly a hug in a bowl.

**Key Ingredients:**

*   **For the Broth:**
    *   Beef bones (knuckle, marrow, oxtail)
    *   Flank steak or brisket (for c

In [2]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents= """List a few asian soups recipes, a yummy description and list the ingredients
        give me fields of
        recipe_name: str
        description: str
        ingredients: list[str]

        not in markdown format    
    """
)

In [3]:
print(response.text)

recipe_name: Tom Yum Goong (Spicy Thai Shrimp Soup)
description: A vibrant and exhilarating Thai hot and sour soup, bursting with aromatic herbs, plump shrimp, and a perfect balance of spicy chili, tangy lime, and savory fish sauce. It's a flavor explosion that will awaken your senses!
ingredients: Shrimp, lemongrass, galangal, kaffir lime leaves, mushrooms, fish sauce, lime juice, chilies (fresh or chili paste), cilantro, water or chicken broth

recipe_name: Japanese Miso Ramen
description: A deeply satisfying and soulful bowl of Japanese ramen featuring a rich, savory miso-infused pork or chicken broth, springy ramen noodles, tender chashu pork, a perfectly soft-boiled egg, and an assortment of delightful toppings that make every spoonful a comforting embrace.
ingredients: Ramen noodles, pork bones or chicken bones (for broth), miso paste, soy sauce, mirin, sake, chashu pork, soft-boiled egg, green onions, nori (seaweed), corn, bean sprouts, sesame oil, garlic, ginger

recipe_name: V

In [4]:
from pydantic import BaseModel

class Recipe(BaseModel):
    recipe_name: str
    description: str
    ingredients: list[str]

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents= "List a few asian soups recipes, a yummy description and list the ingredients",
    config={"response_mime_type": "application/json", "response_schema": list[Recipe]}
)

print(response.text)

[
  {
    "recipe_name": "Miso Soup",
    "description": "A traditional Japanese soup consisting of a dashi broth into which softened miso paste is mixed. It's a comforting and savory soup, often served as a side dish, known for its umami depth and delicate flavor.",
    "ingredients": ["dashi stock", "miso paste", "silken tofu", "wakame seaweed", "green onions"]
  },
  {
    "recipe_name": "Tom Yum Goong (Spicy Shrimp Soup)",
    "description": "A classic Thai hot and sour soup, famous for its vibrant flavors. It's a fragrant broth infused with lemongrass, galangal, kaffir lime leaves, and lime juice, often featuring succulent shrimp and mushrooms.",
    "ingredients": ["shrimp", "lemongrass", "galangal", "kaffir lime leaves", "fish sauce", "lime juice", "mushrooms", "chili paste (nahm prik pao)", "cilantro"]
  },
  {
    "recipe_name": "Pho Bo (Vietnamese Beef Noodle Soup)",
    "description": "A deeply aromatic and comforting Vietnamese noodle soup, featuring a savory beef broth sim

In [6]:
recipes = response.parsed
len(recipes)

3

In [ ]:
recipes[0].description

"A traditional Japanese soup consisting of a dashi broth into which softened miso paste is mixed. It's a comforting and savory soup, often served as a side dish, known for its umami depth and delicate flavor."

In [8]:
recipes[0].recipe_name

'Miso Soup'

In [9]:
recipes[0].ingredients

['dashi stock', 'miso paste', 'silken tofu', 'wakame seaweed', 'green onions']

## simulate house prices

In [15]:
from typing import Literal

class Home(BaseModel):
    price: int
    monthly_fee: int
    living_area: float
    number_rooms: int
    type: Literal["apartment", "house"]
    address: str

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents= """
    List 50 apartments and houses in Stockholm, Sweden, with their address, price, monthly fee, living area, 
    number of rooms, type if it is an apartment or a house. All currencies are in SEK. Data should be simulation of swedish housing.
""",
    config= {"response_mime_type": "application/json", "response_schema": list[Home]}
)



In [16]:
homes = response.parsed
len(homes)

51

In [17]:
homes[0].price, homes[0].address

(5450000, 'Hamngatan 72, Stockholm')

In [20]:
homes[0].__dict__

{'price': 5450000,
 'monthly_fee': 4300,
 'living_area': 62.0,
 'number_rooms': 2,
 'type': 'apartment',
 'address': 'Hamngatan 72, Stockholm'}

In [23]:
import pandas as pd

df = pd.DataFrame([home.__dict__ for home in homes])

In [24]:
df.head()

,price,monthly_fee,living_area,number_rooms,type,address
0,5450000,4300,62.0,2,apartment,"Hamngatan 72, Stockholm"
1,10830000,5800,145.0,5,house,"Valhallavägen 101, Stockholm"
2,4270000,2600,48.0,1,apartment,"Kungsgatan 135, Stockholm"
3,14750000,6200,208.0,7,house,"Drottninggatan 110, Stockholm"
4,3810000,2400,39.0,1,apartment,"Odengatan 23, Stockholm"


In [35]:
chearper_houses = df.query("price < 10_000_000 and type == 'house'")

In [36]:
chearper_houses.to_csv("cheaper_houses.csv", index=False)